In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import json

In [ ]:
# comment and uncomment to run different files

df = pd.read_csv('intermediate/換電站1.csv')
# df = pd.read_csv('intermediate/換電站2.csv')

In [ ]:
vmids = list(df.VmId)

In [ ]:
# empty lists for later use

names = []
addresses = []
hours = []
nBatteries = []
records = []
maintenances = []

for idx, vmid in enumerate(vmids):
    
    print(idx)

    record = []
    maintenance = []

    # scrape the html
    r = requests.get(f'https://mowd.tw/gostation/vmid/{vmid}/')
    r.encoding='utf-8'
    soup = bs(r.text, 'html.parser')

    # find all <h3> tags and <p> tags
    headings = soup.find_all("h3")
    paragraphs = soup.find_all("p")

    # name of the station
    names.append(headings[0].text)

    # first few pieces of information always match
    addresses.append(paragraphs[0].text)
    hours.append(paragraphs[1].text)
    nBatteries.append(paragraphs[2].text)

    # get records
    for paragraph in paragraphs[3:]:
            # check if the current <p> tag is a sibling of <h3> tag
            if paragraph.find_previous_sibling("h3") == headings[1]:
                record.append(paragraph.text)
    records.append(record)

    # find all the sibling of certain tags to get maintenance records
    clearfix = soup.find("div", class_="clearfix")
    for paragraph in paragraphs:
        if paragraph.find_previous_sibling("div") == clearfix:
            maintenance.append(paragraph.text)
    maintenances.append(maintenance)

In [ ]:
# integrate the lists into a dataframe

df2 = pd.DataFrame(
    {'NameOnPage': names,
     'Address': addresses,
     'Hours': hours,
     'nBatteries': nBatteries,
     'Records': records,
     'MaintenanceRecord': maintenances
    })

In [ ]:
# cleaning strings

df2.Address = df2.Address.apply(lambda address: address[3:])
df2.Hours = df2.Hours.apply(lambda hours: hours[5:])
df2.nBatteries = df2.nBatteries.apply(lambda nb: nb[5:nb.index('顆')])
df2.head()

In [ ]:
# merge back to the original dataframe

df_merged = pd.concat([df, df2], axis=1)
df_merged.to_csv('battery_station_2.csv', index=False)